In [183]:
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
####################you will need to change some paths here!#####################
#list of input files
#output files
dir_l2p_start='F:/data/sst/l2p/AMSRE/REMSS/v7/'
dir_l2a='F:/data/sat_data/amsre/L2A/'
dir_l2a_sst='F:/data/sat_data/amsre/L2A_sst/'
#################################################################################
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
from math import floor
import pandas as pd
#from math import cos, radians

In [239]:

for root, dirs, files in os.walk(dir_l2a, topdown=False):
    for name in files:
        if name.endswith(".nc"):
            filename_l2a=os.path.join(root, name)
            name_l2a=name
            with xr.open_dataset(filename_l2a) as l2a_ds:
                iorbit_l2a=floor(l2a_ds.StartOrbitNumber)
                tdim=l2a_ds.Low_Res_Swath_Time.shape[0]
                print(l2a_ds.Low_Res_Swath_Time.shape)
                l2a_time=[0]*tdim
                for i in range(0,tdim):
                    l2a_time[i]=dt.datetime(1993,1,1,0,0,0)+dt.timedelta(seconds=float(l2a_ds.Low_Res_Swath_Time[i].data))
                ilen_l2a=len(l2a_time)
                amsr_year=int(l2a_time[0].year)
                amsr_jdy=int(l2a_time[0].timetuple().tm_yday)
                l2a_sst=np.ones((ilen_l2a,243))*-99.
                l2a_wnd=np.ones((ilen_l2a,243))*-99.
            #now find matching l2p
                dir_l2p=dir_l2p_start+str(amsr_year)+'/'+str(amsr_jdy).zfill(3)+'/'
                print(dir_l2p)
                for root_l2p, dirs_l2p, files_l2p in os.walk(dir_l2p, topdown=False):
                    for name_l2p in files_l2p:
                        print(name_l2a)
                        if name_l2p.endswith(".nc"):
                            filename_l2p=os.path.join(root_l2p, name_l2p)
                            #print(filename_l2p)
                            iorbit=int(name_l2p[-25:-20])
                            if (iorbit==iorbit_l2a) or (iorbit==iorbit_l2a-1) or (iorbit==iorbit_l2a+1):  #same orbit!
                                #print(iorbit,name_l2p) 
                                print(filename_l2p)
                                #amsr_year=int(name[0:4])
                                #amsr_mon=int(name[4:6])
                                #amsr_dym=int(name[6:8])
                                with xr.open_dataset(filename_l2p) as l2p_ds:
                                   # numscan = l2p_ds.sst_dtime.shape[1]
                                   # ncell = l2p_ds.sst_dtime.shape[2]
                                   # l2p_time=[0]*numscan
                                    time0=pd.to_datetime(l2p_ds.time[0].data)
                                    dd=l2p_ds.sst_dtime[0,:,0].data
                                    ilen_l2p=np.argmax(dd) 
                                    l2p_time=[0]*ilen_l2p
                                    for i in range(0,ilen_l2p):
#                                    for i in range(0,numscan):
                                        tem=time0+dt.timedelta(seconds=float(l2p_ds.sst_dtime[0,i,0].data))
                                        l2p_time[i]=tem.to_pydatetime()
#start putting data into l2a sst and wnd
                                    l2p_start=l2p_time[0]
                                    l2p_end=l2p_time[-1]
                                    l2a_start=l2a_time[0]
                                    l2a_end=l2a_time[-1]
                                    ilen_l2a=len(l2a_time)
                                    if l2p_end<l2a_start:
                                        continue
                                    if l2a_start>l2p_end:
                                        continue
                                    #ilen_l2p=len(l2p_time)
                                    imatch=99999
                                    imatchi=-1
                                    if (l2a_start<l2p_start) and (l2a_end<=l2p_end):  #find l2p_start in l2a
                                        for iscan in range(0,ilen_l2a):
                                            dtime=l2p_start-l2a_time[iscan]
                                            if dtime.seconds<imatch:
                                                imatch=dtime.seconds
                                                imatchi=iscan
                                        l2a_sst[imatchi:,:]=l2p_ds.sea_surface_temperature[0,0:ilen_l2a-imatchi,:]    
                                        l2a_wnd[imatchi:,:]=l2p_ds.wind_speed[0,0:ilen_l2a-imatchi,:]  
                                    if (l2a_start>l2p_start) and (l2a_end<=l2p_end):  #find l2a_start in l2p
                                        for iscan in range(0,ilen_l2p):
                                            dtime=l2p_time[iscan]-l2a_start
                                            if dtime.seconds<imatch:
                                                imatch=dtime.seconds
                                                imatchi=iscan
                                        l2a_sst=l2p_ds.sea_surface_temperature[0,imatchi:imatchi+ilen_l2a,:]    
                                        l2a_wnd=l2p_ds.wind_speed[0,imatchi:imatchi+ilen_l2a,:]  
                                    if (l2a_start>l2p_start) and (l2a_end>l2p_end):  #find l2a_start in l2p
                                        for iscan in range(0,ilen_l2p):
                                            dtime=l2p_time[iscan]-l2a_start
                                            if dtime.seconds<imatch:
                                                imatch=dtime.seconds
                                                imatchi=iscan
                                        l2a_sst[:ilen_l2p-imatchi,:]=l2p_ds.sea_surface_temperature[0,imatchi:ilen_l2p,:]    
                                        l2a_wnd[:ilen_l2p-imatchi,:]=l2p_ds.wind_speed[0,imatchi:ilen_l2p,:]  
            #output data
            filename_out_nc=dir_l2a_sst+name_l2a
            fnc = Dataset(filename_out_nc,'w', format='NETCDF4') 
            fnc.createDimension('x', ilen_l2a)
            fnc.createDimension('y', 243)
            s_netcdf = fnc.createVariable('sst', 'f4', ('x','y'))
            w_netcdf = fnc.createVariable('wnd', 'f4', ('x','y'))
            s_netcdf[:] = l2a_sst
            w_netcdf[:] = l2a_wnd
            fnc.close()
    
 




(2002,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101002848-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40758.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010026_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_2010

(1995,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101034608-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40760.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101052520-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40761.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101070432-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40762.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001010612_D.nc
AMSR_E_L2A_Brigh

AMSR_E_L2A_BrightnessTemperatures_V12_201001011109_D.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101120104-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40765.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011109_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011109_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011109_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011109_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011109_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011109_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011109_D.nc
(2000,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/
AMSR_E_L2A_BrightnessTemperatures_V12_201001011158_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011158_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011158_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011158_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011158_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011158_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20

AMSR_E_L2A_BrightnessTemperatures_V12_201001011655_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101151824-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40767.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011655_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101165736-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40768.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011655_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011655_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011655_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011655_A.nc
(1996,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/
AMSR_E_L2A_BrightnessTemperatures_V12_201001011744_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011744_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011744_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011744_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011744_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001011744_D.nc
AMSR_E_L2A_BrightnessTemperat

AMSR_E_L2A_BrightnessTemperatures_V12_201001012241_D.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101215408-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40771.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012241_D.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/20100101233320-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r40772.dat-v02.0-fv01.0.nc
(2000,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2010/001/
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_201001012330_A.nc
AMSR_E_L2A_BrightnessTemperat

...

(2004,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/20090102032240-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r35459.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/20090102050152-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r35460.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/20090102064104-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r35461.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901020638_A.nc
AMSR_E_L2A_Brigh

AMSR_E_L2A_BrightnessTemperatures_V12_200901021134_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/20090102113736-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r35464.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021134_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021134_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021134_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021134_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021134_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021134_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021134_A.nc
(1995,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/
AMSR_E_L2A_BrightnessTemperatures_V12_200901021224_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021224_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021224_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021224_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021224_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021224_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_2009

AMSR_E_L2A_BrightnessTemperatures_V12_200901021721_D.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/20090102163408-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r35467.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021721_D.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/20090102181320-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r35468.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021721_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021721_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021721_D.nc
(2000,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/
AMSR_E_L2A_BrightnessTemperatures_V12_200901021810_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021810_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021810_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021810_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021810_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021810_A.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901021810_A.nc
AMSR_E_L2A_BrightnessTemperat

AMSR_E_L2A_BrightnessTemperatures_V12_200901022307_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/20090102213040-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r35470.dat-v02.0-fv01.0.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022307_A.nc
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/20090102230952-REMSS-L2P_GHRSST-SSTsubskin-AMSRE-l2b_v07a_r35471.dat-v02.0-fv01.0.nc
(1998,)
F:/data/sst/l2p/AMSRE/REMSS/v7/2009/002/
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperatures_V12_200901022356_D.nc
AMSR_E_L2A_BrightnessTemperat

AttributeError: 'Dataset' object has no attribute 'StartOrbitNumber'

In [238]:
print(l2a_start,l2p_start,l2a_end,l2p_end)
print(ilen_l2p,imatchi)
l2a_sst[:ilen_l2p-imatchi,:]=l2p_ds.sea_surface_temperature[0,imatchi:ilen_l2p,:]
l2a_wnd[:ilen_l2p-imatchi,:]=l2p_ds.wind_speed[0,imatchi:ilen_l2p,:]


2010-01-01 02:54:56.989872 2010-01-01 00:28:49 2010-01-01 03:44:44.773552 2010-01-01 02:07:41
4074 0


ValueError: could not broadcast input array from shape (4074,243) into shape (1993,243)

In [194]:
ilen_data=ilen_l2p-imatchi
ilen_data_end=ilen_l2a
if ilen_data>ilen_l2a:
    ilen_data=ilen_l2a
l2a_sst[:ilen_data,:]=l2p_ds.sea_surface_temperature[0,imatchi:ilen_data+imatchi,:]

(1, 4193, 243)
1994
1994 4193 2006


In [230]:
 
numscan = l2p_ds.sst_dtime.shape[1]
ncell = l2p_ds.sst_dtime.shape[2]
l2p_time=[0]*numscan
time0=pd.to_datetime(l2p_ds.time[0].data)
print(numscan,ncell,time0)
print(l2p_ds.sst_dtime[0,500:501,0].data)
print(l2p_ds.sst_dtime[0,4050:4052,0].data)
dd=l2p_ds.sst_dtime[0,:,0].data
ilen_l2p=np.argmax(dd) 
l2p_time=[0]*ilen_l2p
for i in range(0,ilen_l2p):
#    tem=time0+dt.timedelta(seconds=float(l2p_ds.sst_dtime[0,i,0].data))
    tem=time0+dt.timedelta(seconds=float(l2p_ds.sst_dtime[0,i,0].data))
    l2p_time[i]=tem.to_pydatetime()
print(l2p_time[-1])

4193 243 2010-01-01 00:28:49
[573.]
[5898. 5899.]
2010-01-01 02:07:41


In [228]:
print(l2p_ds.sst_dtime[0,4074:4082,0].data)
dd=l2p_ds.sst_dtime[0,:,0].data
imax=np.argmax(dd) 
print(imax)

[5933.    0.    0.    0.    0.    0.    0.    0.]
4074


In [200]:
#l2a_start>l2p_start) and (l2a_end>l2p_end
print(l2a_start,l2p_start,l2a_end,l2p_end)
print(l2p_time[1],l2p_time[-1])
print(numscan)

2010-01-01 01:16:00.415664 2010-01-01 00:28:49 2010-01-01 02:05:49.700784 2010-01-01 00:28:49
2010-01-01 00:28:49 2010-01-01 00:28:49
4193


In [148]:
from math import floor

l2p_start=l2p_time[0]
l2p_end=l2p_time[-1]
l2a_start=l2a_time[0]
l2a_end=l2a_time[-1]
ilen_l2a=len(l2a_time)
ilen_l2p=len(l2p_time)
imatch=99999
imatchi=-1
if (l2a_start<l2p_start) and (l2a_end<=l2p_end):  #find l2p_start in l2a
    for iscan in range(0,ilen_l2a):
        dtime=l2p_start-l2a_time[iscan]
        if dtime.seconds<imatch:
            imatch=dtime.seconds
            imatchi=iscan
    l2a_sst[imatchi:,:]=l2p_ds.sea_surface_temperature[0,0:ilen_l2a-imatchi,:]    
    l2a_wnd[imatchi:,:]=l2p_ds.wind_speed[0,0:ilen_l2a-imatchi,:]  
if (l2a_start>l2p_start) and (l2a_end<=l2p_end):  #find l2a_start in l2p
    for iscan in range(0,ilen_l2p):
        dtime=l2p_start[iscan]-l2a_start
        if dtime.seconds<imatch:
            imatch=dtime.seconds
            imatchi=iscan
    l2a_sst=l2p_ds.sea_surface_temperature[0,imatchi:imatchi+ilen_l2a,:]    
    l2a_wnd=l2p_ds.wind_speed[0,imatchi:imatchi+ilen_l2a,:]  
if (l2a_start>l2p_start) and (l2a_end>l2p_end):  #find l2a_start in l2p
    for iscan in range(0,ilen_l2p):
        dtime=l2p_start[iscan]-l2a_start
        if dtime.seconds<imatch:
            imatch=dtime.seconds
            imatchi=iscan
    l2a_sst[:imatchi,:]=l2p_ds.sea_surface_temperature[0,imatchi:imatchi+ilen_l2a,:]    
    l2a_wnd[:imatchi,:]=l2p_ds.wind_speed[0,imatchi:imatchi+ilen_l2a,:]  
    
    


86260
stop!
40758 40757.727 40758.23
1089


In [172]:
#print(l2p_time[0]-l2a_time[istart_scan-996])
#print(l2p_ds.sea_surface_temperature.shape)
ilen_l2a=len(l2a_time)
ilen_l2p=len(l2p_time)
imatch=99999
imatchi=-1
l2a_sst=np.zeros((ilen_l2a,243))
l2a_wnd=np.zeros((ilen_l2a,243))
for iscan in range(0,len(l2a_time)):
    dtime=l2p_time[0]-l2a_time[iscan]
    #print(dtime.seconds,imatch)
    if dtime.seconds<imatch:
        imatch=dtime.seconds
        imatchi=iscan
if imatchi>=0:
    #for iscan in range(imatchi,len(l2a_time))
    l2a_sst[imatchi:,:]=l2p_ds.sea_surface_temperature[0,0:ilen_l2a-imatchi,:]    
    l2a_wnd[imatchi:,:]=l2p_ds.wind_speed[0,0:ilen_l2a-imatchi,:]    


0:00:00.380417
(1, 4193, 243)


In [177]:

print(l2a_wnd[500,:])
#print(len(l2a_time))
#print(type(l2p_time[0]))
#print('match:', imatch,imatchi)
#print(type(ilen_l2a),type(imatchi))
    
#print(l2p_time[0]-l2a_time[0])
#dt=l2a_time-l2p_time[0]
#timei = np.argmin( np.abs( l2a_time - l2p_time[0] ) )
#dttime=abs(l2p_time[0]-l2a_time)
#dttime=abs(time_ccmp2[timei] - amsr_date)
 #   f2=dttime.seconds/(6.*60*60)


[ 3.79999924  4.          4.19999886  4.19999886  4.19999886  4.39999962
  4.79999924  5.19999886  5.39999962  5.19999886  5.19999886  5.39999962
  5.19999886  4.79999924  4.59999847  4.59999847  4.59999847  5.
  5.19999886  5.19999886  5.39999962  6.          6.39999962  6.59999847
  6.59999847  6.59999847  7.          7.19999886  7.39999962  7.39999962
  7.39999962  8.          8.59999847  8.39999962  8.19999886  8.19999886
  8.39999962  8.          7.59999847  7.59999847  8.          8.
  7.79999924  7.79999924  8.          8.19999886  8.39999962  7.79999924
  7.39999962  7.59999847  8.19999886  8.19999886  8.59999847  9.
  9.19999886  9.          8.79999924  9.39999962  9.59999943  9.39999962
  9.39999962  9.19999886  9.59999943  9.99999905  9.99999905  9.39999962
  9.59999943  9.99999905  9.99999905  9.99999905  9.99999905  9.99999905
  9.39999962  9.          9.          9.39999962  9.59999943  9.39999962
  9.39999962  9.79999924 10.39999962 10.59999943 10.59999943 10.59999943
 1

In [181]:
int(l2a_time[0].timetuple().tm_yday)
print(l2a_ds.StartOrbitNumber)

40757.727


In [18]:
#print(l2a_ds.StartOrbitNumber)
#l2a_date=l2a_ds.RangeBeginningDate
#l2a_time=l2a_ds.RangeBeginningTime
#print(iorbit)
#print(l2p_ds.start_time)
#print(l2a_ds.Position_in_Orbit[0:10].data)
#print(l2a_ds.Hot_Load_Count_6_to_52.shape) #Effective_Hot_Load_Temperature_6_to_52
#print(l2a_ds.Low_Res_Swath_Time[0:10].data) #seconds since 1993-01-01T00:00:00

[40757.72492051 40757.72517332 40757.72542617 40757.72567896
 40757.72593178 40757.72618455 40757.72643738 40757.7266901
 40757.72694286 40757.72719556]
(2002, 16, 12)
[5.36459189e+08 5.36459191e+08 5.36459192e+08 5.36459194e+08
 5.36459195e+08 5.36459197e+08 5.36459198e+08 5.36459200e+08
 5.36459201e+08 5.36459203e+08]


In [82]:
l2a_time[0:2]

[datetime.datetime(2010, 1, 1, 0, 26, 29, 129136),
 datetime.datetime(2010, 1, 1, 0, 26, 29, 129136)]

In [91]:
print(type(l2p_time[200]))
type(l2p_time[200].to_pydatetime())

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


datetime.datetime

In [80]:
#    print(l2p_time[i],l2p_ds.sst_dtime[0,i,0].data)


In [ ]:
timei = np.argmin( np.abs( time_ccmp2 - amsr_date ) )
dttime=abs(time_ccmp2[timei] - amsr_date)
f2=dttime.seconds/(6.*60*60)


In [ ]:
idysv=0
istart=0
col_wndu=[0]*tdim
col_wndv=[0]*tdim
for i in range(0,tdim):
    amsr_date=date_1993+dt.timedelta(seconds=float(amsr_dysince[i].values))  #create new time array that can be queried for year etc
    #print(i,amsr_date,idysv)
    if istart==0: #initialize data
        for incr in range(-1,2):
            amsr_date2=amsr_date+dt.timedelta(days=float(incr))  #create new time array that can be queried for year etc
            syr=str(amsr_date2.year).zfill(4)
            smon=str(amsr_date2.month).zfill(2)
            sdym=str(amsr_date2.day).zfill(2)
            sjdy=str(amsr_date2.timetuple().tm_yday).zfill(3)
            fname_tem='/CCMP_Wind_Analysis_' + syr + smon + sdym + '_V02.0_L3.0_RSS.nc'
            ccmp_filename = dir_ccmp + syr + '/M' + smon + fname_tem      
            print(ccmp_filename)
            nc_fid = Dataset(ccmp_filename, 'r')
            tem = nc_fid.variables['uwnd'][:]
            wndu = np.append(tem[:,720:],tem[:,:720], axis=1)
            tem = nc_fid.variables['vwnd'][:]
            wndv = np.append(tem[:,720:],tem[:,:720], axis=1)               
            mlat_ccmp = nc_fid.variables['latitude'][:]
            tem = nc_fid.variables['longitude'][:]
            mlon_ccmp = np.append(tem[720:],tem[:720], axis=0)               
            mlon_ccmp = ((mlon_ccmp - 180) % 360) - 180  #make -180 to 180 rather than 0 360
            t=nc_fid.variables['time'][:] #units: hours since 1987-01-01 00:00:00
            time_ccmp=[0]*4
            for itt in range(0,4):
                time_ccmp[itt]=dt.datetime(1987,1,1,0,0,0)+dt.timedelta(hours=t[itt])
            nc_fid.close()
            if incr==-1:
                wndu2=wndu
                wndv2=wndv
                time_ccmp2=time_ccmp
            else:
                wndu2 =  np.append(wndu2,wndu, axis=0)
                wndv2 =  np.append(wndv2,wndv, axis=0)
                time_ccmp2 = np.append(time_ccmp2,time_ccmp, axis = 0)
            idysv=amsr_date.day
            istart=1
    if amsr_date.day!=idysv:
        #print('read in new data',amsr_date.day,idysv)
        amsr_date2=amsr_date+dt.timedelta(days=float(1))  #create new time array that can be queried for year etc
        syr=str(amsr_date2.year).zfill(4)
        smon=str(amsr_date2.month).zfill(2)
        sdym=str(amsr_date2.day).zfill(2)
        sjdy=str(amsr_date2.timetuple().tm_yday).zfill(3)
        fname_tem='/CCMP_Wind_Analysis_' + syr + smon + sdym + '_V02.0_L3.0_RSS.nc'
        ccmp_filename = dir_ccmp + syr + '/M' + smon + fname_tem      
        print(ccmp_filename)
        nc_fid = Dataset(ccmp_filename, 'r')
        tem = nc_fid.variables['uwnd'][:]
        wndu = np.append(tem[:,720:],tem[:,:720], axis=1)               
        tem = nc_fid.variables['vwnd'][:]
        wndv = np.append(tem[:,720:],tem[:,:720], axis=1)               
        mlat_ccmp = nc_fid.variables['latitude'][:]
        tem = nc_fid.variables['longitude'][:]
        mlon_ccmp = np.append(tem[720:],tem[:720], axis=0)               
        mlon_ccmp = ((mlon_ccmp - 180) % 360) - 180  #make -180 to 180 rather than 0 360
        t=nc_fid.variables['time'][:] #units: hours since 1987-01-01 00:00:00
        time_ccmp=[0]*4
        for itt in range(0,4):
            time_ccmp[itt]=dt.datetime(1987,1,1,0,0,0)+dt.timedelta(hours=t[itt])
        nc_fid.close()
        idysv=amsr_date.day
        wndu2[0:8,:,:]=wndu2[4:12,:,:]
        wndv2[0:8,:,:]=wndv2[4:12,:,:]
        time_ccmp2[0:8]=time_ccmp2[4:12]
        wndu2[8:12,:,:]=wndu[:]
        wndv2[8:12,:,:]=wndv[:]
        time_ccmp2[8:12]=time_ccmp[:]
    latli = np.argmin( np.abs( mlat_ccmp - lats_amsr[i] ) )
    lonli = np.argmin( np.abs( mlon_ccmp - lons_amsr[i] ) )
    timei = np.argmin( np.abs( time_ccmp2 - amsr_date ) )
    dttime=abs(time_ccmp2[timei] - amsr_date)
    f2=dttime.seconds/(6.*60*60)
    f1=f2-1.
    if time_ccmp2[timei]<amsr_date:
        timej=timei+1
    if time_ccmp2[timei]>=amsr_date:
        timej=timei-1
    #print(f1,f2,timei,timej)
    col_wndu[i]=f1*wndu2[timei,latli,lonli]+f2*wndu2[timej,latli,lonli]
    col_wndv[i]=f1*wndv2[timei,latli,lonli]+f2*wndv2[timej,latli,lonli]
    
                


F:/data/sat_data/ccmp/v02.0/Y2009/M12/CCMP_Wind_Analysis_20091230_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2009/M12/CCMP_Wind_Analysis_20091231_V02.0_L3.0_RSS.nc
F:/data/sat_data/ccmp/v02.0/Y2010/M01/CCMP_Wind_Analysis_20100101_V02.0_L3.0_RSS.nc
0 output np.save
F:/data/sat_data/ccmp/v02.0/Y2010/M01/CCMP_Wind_Analysis_20100102_V02.0_L3.0_RSS.nc


In [ ]:
print(wndu2[:,300,700])

In [ ]:
Dataset(ccmp_filename).variables['time']

In [ ]:
#Dataset(ccmp_filename).variables['drifter___sst_insitu__time']